<a href="https://colab.research.google.com/github/Izuho/senior-thesis/blob/main/Qiskit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit
!pip install platexenc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 6.2 MB/s 
     |████████████████████████████████| 12.8 MB 53.0 MB/s 
     |████████████████████████████████| 240 kB 13.4 MB/s 
     |████████████████████████████████| 106 kB 51.6 MB/s 
     |████████████████████████████████| 55 kB 1.1 MB/s 
     |████████████████████████████████| 49 kB 3.8 MB/s 
     |████████████████████████████████| 929 kB 52.1 MB/s 
     |████████████████████████████████| 37.5 MB 13.6 MB/s 
     |████████████████████████████████| 50 kB 3.4 MB/s 
     |████████████████████████████████| 4.0 MB 50.7 MB/s 
     |████████████████████████████████| 1.9 MB 46.4 MB/s 
     |████████████████████████████████| 112 kB 55.9 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.4-py3-none-any.whl size=12274 sha256=cf98e59592d41bdd61de21bb2ca9ea773364462a15557a8ba5e1cda590440eaf
  Stored in directory: /root/.cache/pip/wheels/40/2c/d7/aa0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
X1 = np.load('drive/MyDrive/mnist_train_feat.npy')
y1 = np.load('drive/MyDrive/mnist_train_Label.npy')
X2 = np.load('drive/MyDrive/mnist_test_feat.npy')
y2 = np.load('drive/MyDrive/mnist_test_Label.npy')
print(X1.shape, y1.shape, X2.shape, y2.shape)

(12665, 64) (12665,) (2115, 64) (2115,)


In [4]:
from qiskit.circuit import Parameter, QuantumCircuit

def fraxis_gate(nx, ny, nz=None):
    nx, ny, nz = _validate(nx, ny, nz)
    theta = 2*(np.arccos(nz))
    phi = None
    if nx==0:
      phi=np.pi/2
    else:
      phi=np.arctan2(ny, nx)
    circ = QuantumCircuit(1)
    
    circ.u(theta,phi,np.pi-phi,[0])
    return circ

def _validate(nx, ny, nz=None):
    assert nx**2+ny**2<=1, print(nx,ny)
    if nz == None:
        nx = nx.real
        ny = ny.real
        nz2 = 1-nx**2.-ny**2.
        nz = np.sqrt(nz2).real if nz2>0 else 0
    return nx, ny, nz

In [5]:
def FraxisFeatureMap(num_qubits, data):
    circ = QuantumCircuit(num_qubits)
    for i in range(
        data.shape[0]//2
    ):
        circ.compose(fraxis_gate(data[2*i], data[2*i+1]), qubits=[i%num_qubits], inplace=True)
        if (i+1)%num_qubits == 0:
            for j in range(0,num_qubits,2):
                if j+1 < num_qubits:
                    circ.cz(j,j+1)
            for j in range(1,num_qubits,2):
                if j+1 < num_qubits:
                    circ.cz(j,j+1)        
    return circ

In [6]:
def FraxisAnsatz(num_qubits, params):
    circ = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        circ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
    for j in range(0,num_qubits,2):
        if j+1 < num_qubits:
            circ.cz(j,j+1)
    for j in range(1,num_qubits,2):
        if j+1 < num_qubits:
            circ.cz(j,j+1)        
    return circ

In [7]:
def replace_FraxisAnsatz(num_qubits, target, params):
    circX = QuantumCircuit(num_qubits)
    circY = QuantumCircuit(num_qubits)
    circZ = QuantumCircuit(num_qubits)
    circXY = QuantumCircuit(num_qubits)
    circXZ = QuantumCircuit(num_qubits)
    circYZ = QuantumCircuit(num_qubits)
    for i in range(target):
        circX.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circYZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
    circX.x(target)
    circY.y(target)
    circZ.z(target)
    circXY.u(np.pi, np.pi*0.25, np.pi*0.75, target)
    circXZ.u(np.pi*0.5, 0, np.pi, target)
    circYZ.u(np.pi*0.5, np.pi*0.5, np.pi*0.5, target)
    for i in range(target+1, num_qubits, 1):
        circX.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circYZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
    for j in range(0,num_qubits,2):
        if j+1 < num_qubits:
            circX.cz(j,j+1)
            circY.cz(j,j+1)
            circZ.cz(j,j+1)
            circXY.cz(j,j+1)
            circXZ.cz(j,j+1)
            circYZ.cz(j,j+1)
    for j in range(1,num_qubits,2):
        if j+1 < num_qubits:
            circX.cz(j,j+1)
            circY.cz(j,j+1)
            circZ.cz(j,j+1)
            circXY.cz(j,j+1)
            circXZ.cz(j,j+1)
            circYZ.cz(j,j+1)
    return [circX, circY, circZ, circXY, circXZ, circYZ]

In [8]:
from qiskit.primitives import Sampler
from qiskit.circuit import ClassicalRegister

class FraxClassify_sim():
    def __init__(self, n_qubits, layer_size, world_size, service):
        self.n_qubits = n_qubits
        self.layer_size = layer_size
        self.params = np.zeros((layer_size, n_qubits, 2))
        #np.zeros((layer_size, n_qubits, 2))
        #np.random.rand(layer_size, n_qubits, 2)*2/3
        #np.zeros((layer_size, n_qubits, 2)) + 1/np.sqrt(3)
        self.world_size =world_size
        self.service = service
    def eval(self, X, y, sampler):
        acc, score= 0,0
        for a in range(X.shape[0]//self.world_size):
            qc = QuantumCircuit(self.n_qubits*self.world_size)
            feature_map = []
            for b in range(self.world_size):
                feature_map.append(FraxisFeatureMap(self.n_qubits, X[a*self.world_size+b]))
            for b in range(self.layer_size):
                for c in range(self.world_size):
                    qc.compose(
                        feature_map[c], 
                        qubits=range(self.n_qubits*c,self.n_qubits*(c+1),1), 
                        inplace=True)
                    qc.compose(
                        FraxisAnsatz(self.n_qubits, self.params[b]), 
                        qubits=range(self.n_qubits*c,self.n_qubits*(c+1),1), 
                        inplace=True
                    )
            
            cr = ClassicalRegister(self.world_size)
            qc.add_register(cr)
            qc.measure(
                range(0,self.world_size*self.n_qubits,self.n_qubits),
                cr
            )
                    
            result = sampler.run(
                circuits=[qc],
            ).result().quasi_dists
            Zexp = np.zeros(self.world_size)
            for bits in result[0]:
                for e in range(self.world_size):
                    if (bits>>(e))%2 == 1:
                        Zexp[e] -= result[0][bits]
                    else:
                        Zexp[e] += result[0][bits]
                        
            acc += np.sum(np.where(y[self.world_size*a:self.world_size*(a+1)]*Zexp>0,1,0))
            score += np.sum(y[self.world_size*a:self.world_size*(a+1)]*Zexp)
        #print(acc, score*2)
        return acc, score*2
        
    def fit_and_eval(self, X, y, X2, y2):
        sampler = Sampler()
        for a in range(self.layer_size):
            for b in range(self.n_qubits):
                R = np.zeros((3,3))
                for c in range(y.shape[0]//self.world_size):
                    feature_map = []
                    for d in range(self.world_size):
                        feature_map.append(FraxisFeatureMap(self.n_qubits, X[c*self.world_size+d]))
                    qcs = []
                    for d in range(6):
                        qcs.append(QuantumCircuit(self.n_qubits*self.world_size))
                    for d in range(a):
                        for e in range(6):
                            for f in range(self.world_size):
                                qcs[e].compose(
                                    feature_map[f], 
                                    qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                    inplace=True
                                )
                            
                        original_ansatz = FraxisAnsatz(self.n_qubits, self.params[d])
                        for e in range(6):
                            for f in range(self.world_size):
                                qcs[e].compose(
                                    original_ansatz, 
                                    qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                    inplace=True
                                )

                    for d in range(6):
                        for f in range(self.world_size):
                            qcs[d].compose(
                                feature_map[f], 
                                qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                inplace=True
                            )
                        
                    ansatzs = replace_FraxisAnsatz(self.n_qubits, b, self.params[a])
                    for d in range(6):
                        for f in range(self.world_size):
                            qcs[d].compose(
                                ansatzs[d], 
                                qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                inplace=True
                            )
                                        
                    for d in range(a+1,self.layer_size,1):
                        for e in range(6):
                            for f in range(self.world_size):
                                qcs[e].compose(
                                    feature_map[f], 
                                    qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                    inplace=True
                                )
                            
                        original_ansatz = FraxisAnsatz(self.n_qubits, self.params[d])
                        for e in range(6):
                            for f in range(self.world_size):
                                qcs[e].compose(
                                    original_ansatz, 
                                    qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                    inplace=True
                                )

                    for d in range(6):
                        cr = ClassicalRegister(self.world_size)
                        qcs[d].add_register(cr)
                        qcs[d].measure(
                            range(0,self.world_size*self.n_qubits,self.n_qubits),
                            cr
                        )
                    result = sampler.run(
                        circuits=qcs,
                    ).result().quasi_dists

                    r6s = np.zeros((6,self.world_size))
                    for d in range(6):
                        #print(result[d])
                        for bits in result[d]:
                            for e in range(self.world_size):
                                if (bits>>(e))%2 == 1:
                                    r6s[d, e] -= result[d][bits]
                                else:
                                    r6s[d, e] += result[d][bits]
                                    
                    R[0,0] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * 2 * r6s[0])
                    R[0,1] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * (2 * r6s[3] - r6s[0] - r6s[1]))
                    R[0,2] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * (2 * r6s[4] - r6s[0] - r6s[2]))
                    R[1,1] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * 2 * r6s[1])
                    R[1,2] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * (2 * r6s[5] - r6s[1] - r6s[2]))
                    R[2,2] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * 2 * r6s[2])
                R[1,0] = R[0,1]
                R[2,0] = R[0,2]
                R[2,1] = R[1,2]

                eigenvalues, eigenvectors = np.linalg.eigh(R)
                
                self.params[a,b,:] = eigenvectors[0:2,np.argmax(eigenvalues)]
                
                print(np.max(eigenvalues))

                acc1, sc1 = self.eval(X,y,sampler)
                if abs(np.max(eigenvalues)-sc1)>1e-4:
                    self.params[a,b,:] *= -1
                    acc1, sc1 = self.eval(X,y,sampler) 
                print('ACC_train: ',acc1,'\nSCORE_train: ',sc1)
                
                acc1, sc1 = self.eval(X2,y2,sampler)
                print('ACC_test: ',acc1,'\nSCORE_test: ',sc1)
        print(self.params)

In [10]:
M=100
N=100

In [ ]:
import time

model = FraxClassify_sim(2, 1, 1, None)
for i in range(20):
    st = time.time()
    model.fit_and_eval(X1[0:M], y1[0:M], X2[0:N], y2[0:N])
    print(time.time()-st)
    print('_______________NEW________________',i)

87.3049239898365
ACC_train:  91 
SCORE_train:  87.30492398983652
ACC_test:  83 
SCORE_test:  70.61479067879786
87.30492398983652
ACC_train:  91 
SCORE_train:  87.30492398983652
ACC_test:  83 
SCORE_test:  70.61479067879786
[[[-0.7891381   0.55790957]
  [ 1.          0.        ]]]
19.55547022819519
_______________NEW________________ 0
87.3049239898365
ACC_train:  91 
SCORE_train:  87.3049239898365
ACC_test:  83 
SCORE_test:  70.61479067879785
87.3049239898365
ACC_train:  91 
SCORE_train:  87.3049239898365
ACC_test:  83 
SCORE_test:  70.61479067879785
[[[-0.7891381   0.55790957]
  [ 1.          0.        ]]]
17.39359736442566
_______________NEW________________ 1
87.3049239898365
ACC_train:  91 
SCORE_train:  87.3049239898365
ACC_test:  83 
SCORE_test:  70.61479067879785
87.3049239898365
ACC_train:  91 
SCORE_train:  87.3049239898365
ACC_test:  83 
SCORE_test:  70.61479067879785
[[[-0.7891381   0.55790957]
  [ 1.          0.        ]]]
17.629585027694702
_______________NEW________________